Import Data

In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
from rapidfuzz import process, fuzz

import warnings
warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

Load Dataset

In [80]:

# access data db
def get_data(query):
    conn = sqlite3.connect("olist.db")
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    return data

In [81]:
def get_df(query):
    # get data
    conn = sqlite3.connect("olist.db")
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()

    # to dataframe
    col_names = [description[0] for description in cursor.description]
    df = pd.DataFrame(data, columns=col_names)
    conn.close()

    return df

EDA dan Data Preprocessing

#1 Informasi dataset

In [82]:
query = """
SELECT 
    name 
FROM 
    sqlite_master 
WHERE type='table'
"""

tables = get_data(query)
tables

[('olist_order_customer_dataset',),
 ('olist_order_dataset',),
 ('olist_order_reviews_dataset',),
 ('olist_order_payments_dataset',),
 ('olist_order_items_dataset',),
 ('olist_products_dataset',),
 ('olist_sellers_dataset',),
 ('olist_geolocation_dataset',),
 ('product_category_name_translation',)]

In [83]:
query = """
SELECT
    *
FROM 
    olist_order_dataset
"""
df_order_dataset = get_df(query)

df_order_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   index                          99441 non-null  int64 
 1   order_id                       99441 non-null  object
 2   customer_id                    99441 non-null  object
 3   order_status                   99441 non-null  object
 4   order_purchase_timestamp       99441 non-null  object
 5   order_approved_at              99281 non-null  object
 6   order_delivered_carrier_date   97658 non-null  object
 7   order_delivered_customer_date  96476 non-null  object
 8   order_estimated_delivery_date  99441 non-null  object
dtypes: int64(1), object(8)
memory usage: 6.8+ MB


In [84]:
query = """
SELECT
    *
FROM 
    olist_order_items_dataset
"""
df_order_items = get_df(query)

df_order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 8 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   index                112650 non-null  int64  
 1   order_id             112650 non-null  object 
 2   order_item_id        112650 non-null  int64  
 3   product_id           112650 non-null  object 
 4   seller_id            112650 non-null  object 
 5   shipping_limit_date  112650 non-null  object 
 6   price                112650 non-null  float64
 7   freight_value        112650 non-null  float64
dtypes: float64(2), int64(2), object(4)
memory usage: 6.9+ MB


In [85]:
query = """
SELECT
    *
FROM 
    olist_order_reviews_dataset
"""
df_order_reviews_dataset = get_df(query)

df_order_reviews_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   index                    99224 non-null  int64 
 1   review_id                99224 non-null  object
 2   order_id                 99224 non-null  object
 3   review_score             99224 non-null  int64 
 4   review_comment_title     11568 non-null  object
 5   review_comment_message   40977 non-null  object
 6   review_creation_date     99224 non-null  object
 7   review_answer_timestamp  99224 non-null  object
dtypes: int64(2), object(6)
memory usage: 6.1+ MB


In [86]:
query = """
SELECT
    *
FROM 
    olist_order_payments_dataset
"""
df_order_payments_dataset = get_df(query)

df_order_payments_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   index                 103886 non-null  int64  
 1   order_id              103886 non-null  object 
 2   payment_sequential    103886 non-null  int64  
 3   payment_type          103886 non-null  object 
 4   payment_installments  103886 non-null  int64  
 5   payment_value         103886 non-null  float64
dtypes: float64(1), int64(3), object(2)
memory usage: 4.8+ MB


In [87]:
query = """
SELECT
    *
FROM 
    olist_products_dataset
"""
df_products_dataset = get_df(query)

df_products_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   index                       32951 non-null  int64  
 1   product_id                  32951 non-null  object 
 2   product_category_name       32341 non-null  object 
 3   product_name_lenght         32341 non-null  float64
 4   product_description_lenght  32341 non-null  float64
 5   product_photos_qty          32341 non-null  float64
 6   product_weight_g            32949 non-null  float64
 7   product_length_cm           32949 non-null  float64
 8   product_height_cm           32949 non-null  float64
 9   product_width_cm            32949 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 2.5+ MB


In [88]:
query = """
SELECT
    *
FROM 
    product_category_name_translation
"""
df_product_category_name_translation = get_df(query)

df_product_category_name_translation.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   index                          71 non-null     int64 
 1   product_category_name          71 non-null     object
 2   product_category_name_english  71 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.8+ KB


#2 Data Wragling

Objective
1. Produk yang populer, banyak terjual, dan menghasilkan revenue terbanyak
2. Produk yang sering dibatalkan customer
3. Review Katgeori Produk terbaik dan terburuk dan proporsi review
4. Kategori Produk dengan installment 1 bulan terbanyak dan payment types proportion

In [89]:
query = """
SELECT
    oi.order_id,
    p.product_id,
    t.product_category_name_english AS product_name,
    oi.price,
    op.payment_installments,
    op.payment_type,
    o.order_status,
    r.review_score    
FROM 
    olist_order_items_dataset oi
LEFT JOIN 
    olist_products_dataset p ON oi.product_id = p.product_id
LEFT JOIN 
    product_category_name_translation t ON p.product_category_name = t.product_category_name
LEFT JOIN 
    olist_order_dataset o ON oi.order_id = o.order_id
LEFT JOIN
    olist_order_reviews_dataset r ON oi.order_id = r.order_id
LEFT JOIN
    olist_order_payments_dataset op ON oi.order_id = op.order_id
"""

df = get_df(query)
df

,order_id,product_id,product_name,price,payment_installments,payment_type,order_status,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.90,2.0,credit_card,delivered,5.0
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,pet_shop,239.90,3.0,credit_card,delivered,4.0
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,furniture_decor,199.00,5.0,credit_card,delivered,5.0
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,perfumery,12.99,2.0,credit_card,delivered,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,garden_tools,199.90,3.0,credit_card,delivered,5.0
...,...,...,...,...,...,...,...,...
118305,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,housewares,299.99,1.0,boleto,delivered,5.0
118306,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,computers_accessories,350.00,1.0,boleto,delivered,5.0
118307,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,sports_leisure,99.90,3.0,credit_card,delivered,5.0
118308,fffe18544ffabc95dfada21779c9644f,9c422a519119dcad7575db5af1ba540e,computers_accessories,55.99,3.0,credit_card,delivered,5.0


In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118310 entries, 0 to 118309
Data columns (total 8 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              118310 non-null  object 
 1   product_id            118310 non-null  object 
 2   product_name          116576 non-null  object 
 3   price                 118310 non-null  float64
 4   payment_installments  118307 non-null  float64
 5   payment_type          118307 non-null  object 
 6   order_status          118310 non-null  object 
 7   review_score          117332 non-null  float64
dtypes: float64(3), object(5)
memory usage: 7.2+ MB


#3 Data Cleaning

In [91]:
df_desc = []

for i in df.columns:
    df_desc.append([
        i,
        df[i].dtypes,
        df[i].isna().sum(),
        round((((df[i].isna().sum()) / (len(df))) * 100),2),
        df[i].nunique(),
        df[i].drop_duplicates().sample(2).values
    ])

df_analysis = pd.DataFrame(df_desc, columns = ['Data Features', 'Data Type', 'Null', 'Null (%)', 'Unique', 'Unique Sample'])
df_analysis

,Data Features,Data Type,Null,Null (%),Unique,Unique Sample
0,order_id,object,0,0.00,98666,"[7eb5250ca7b7e2436c5114888c655c1d, 2ba265d6ef8..."
1,product_id,object,0,0.00,32951,"[817e7f7aeb6d2edd3000b35a1de238f0, 6f7de978459..."
2,product_name,object,1734,1.47,71,"[housewares, fashion_underwear_beach]"
3,price,float64,0,0.00,5968,"[34.47, 196.85]"
4,payment_installments,float64,3,0.00,24,"[13.0, 14.0]"
5,payment_type,object,3,0.00,4,"[None, credit_card]"
6,order_status,object,0,0.00,7,"[delivered, shipped]"
7,review_score,float64,978,0.83,5,"[2.0, 3.0]"


#3.1 Handling Duplicates

In [93]:
df_duplicates = df.duplicated().sum()
df_duplicates

13174

In [94]:
#drop duplicates
df.drop_duplicates(inplace=True)
df

,order_id,product_id,product_name,price,payment_installments,payment_type,order_status,review_score
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,cool_stuff,58.90,2.0,credit_card,delivered,5.0
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,pet_shop,239.90,3.0,credit_card,delivered,4.0
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd,furniture_decor,199.00,5.0,credit_card,delivered,5.0
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,perfumery,12.99,2.0,credit_card,delivered,4.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,garden_tools,199.90,3.0,credit_card,delivered,5.0
...,...,...,...,...,...,...,...,...
118305,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0,housewares,299.99,1.0,boleto,delivered,5.0
118306,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,computers_accessories,350.00,1.0,boleto,delivered,5.0
118307,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482,sports_leisure,99.90,3.0,credit_card,delivered,5.0
118308,fffe18544ffabc95dfada21779c9644f,9c422a519119dcad7575db5af1ba540e,computers_accessories,55.99,3.0,credit_card,delivered,5.0


Duplicates berhasil di bersihkan

#3.2 Handling Missing Values

In [95]:
#Karena jumlah nilai missing value sangat sedikit, yaitu 1.47%. Jadi Kita akan mendrop nya
df.dropna(inplace=True)

Pada kolom product_name hanya memiliki null sebesar 1,44% yang mana dikategorikan sangat kecil sehingga tidak memberikan efek signifikan pada dataset. Serta untuk menghindari bias pada analisis jika ingin melakukan analisis berupa pengkategorian pada product_name

#3.3 Handling Inconsistent Format